In [2]:
import numpy as np
import re
import requests
import io
import yaml

from glob import glob
from os.path import exists, join, basename
from bs4 import BeautifulSoup
from shutil import copyfileobj, copyfile
from PIL import Image
from time import sleep
from requests_ip_rotator import ApiGateway, EXTRA_REGIONS, ALL_REGIONS
from tqdm import tqdm

## Package Screenshot

In [169]:
with open('./Interesting.html', 'r') as fp:
    content = fp.read()
    soup = BeautifulSoup(content, 'html.parser')
    
table = soup.find('table')
rows = table.find_all('tr')


In [171]:
gateway = ApiGateway("https://lh3.googleusercontent.com")
gateway.start()

session = requests.Session()
session.mount("https://lh3.googleusercontent.com", gateway)

Starting API gateways in 10 regions.
Using 10 endpoints with name 'https://lh3.googleusercontent.com - IP Rotate API' (10 new).


In [188]:
gateway.shutdown()

Deleting gateways for site 'https://lh3.googleusercontent.com'.
Deleted 10 endpoints with for site 'https://lh3.googleusercontent.com'.


['tsm1w4s5fl',
 'nyneblenn4',
 'gczc70jn89',
 'tw3a1u6385',
 'p0tbo4h6md',
 'glmj64pmcl',
 '6elbecj32k',
 '60z8214b31',
 'co86x63i1a',
 'wikndn9lqi']

In [189]:
# for i, row in enumerate(columns):
#     print(i, row)

In [187]:
existing_files = set([basename(f).replace('.webp', '') for f in glob('./thumbnails/*.webp')])

for i, row in tqdm(enumerate(rows)):

    columns = row.find_all('td')
    if len(columns) < 1:
        continue

    name = columns[0]
    if name is None:
        continue
    
    name = name.text.lower().replace(' ', '-')
    name = name.split('.')[0]
    
    if name in existing_files or name == 'module' or name == '':
        continue
    
    # Remove the size query from image_src
    image_src = columns[19].find('img').get('src')
    image_src = re.sub(r'^(.*)=w\d+-h\d+$', r'\1', image_src)
    
    # Save the image as webp
    response = session.get(image_src, stream=True)

    if response.status_code == 200:
        image = Image.open(io.BytesIO(response.content))
        image_file_path = f'./thumbnails/{name}.webp'
        image.save(image_file_path, lossless=False, quality=80)
        sleep(0.5)
    else:
        print(response.status_code, response, name)

105it [00:00, 59586.24it/s]


## Paper Screenshot

In [7]:
with open('./Papers.html', 'r') as fp:
    content = fp.read()
    soup = BeautifulSoup(content, 'html.parser')
    
table = soup.find('table')
rows = table.find_all('tr')

In [4]:
gateway = ApiGateway("https://lh3.googleusercontent.com")
gateway.start()

session = requests.Session()
session.mount("https://lh3.googleusercontent.com", gateway)

Starting API gateways in 10 regions.
Using 10 endpoints with name 'https://lh3.googleusercontent.com - IP Rotate API' (10 new).


In [196]:
# row = rows[5]
# columns = row.find_all('td')
# for i, r in enumerate(columns):
#     print(i, r)

In [9]:
existing_files = set([basename(f).replace('.webp', '') for f in glob('./thumbnails/*.webp')])

for i, row in tqdm(enumerate(rows), total=len(rows)):
    columns = row.find_all('td')
    
    if len(columns) < 1:
        continue
    
    name = columns[0]
    if name is None:
        continue
    
    name = name.text.lower().replace(' ', '-')
    name = name.split('.')[0]
    
    if name in existing_files or name == 'name' or name == '':
        continue
    
    # Remove the size query from image_src
    image = columns[20]
    
    if image is None:
        continue

    image_src = image.find('img').get('src')
    image_src = re.sub(r'^(.*)=w\d+-h\d+$', r'\1', image_src)
    
    # Save the image as webp
    response = session.get(image_src, stream=True)

    if response.status_code == 200:
        image = Image.open(io.BytesIO(response.content))
        image_file_path = f'./thumbnails/{name}.webp'
        image.save(image_file_path, lossless=False, quality=80)
        sleep(0.5)
    else:
        print(response.status_code, response, name)


100%|██████████| 67/67 [00:01<00:00, 42.24it/s]


In [10]:
gateway.shutdown()

Deleting gateways for site 'https://lh3.googleusercontent.com'.
Deleted 10 endpoints with for site 'https://lh3.googleusercontent.com'.


['t5gq2nzgqj',
 'pxdvfoojgk',
 'snm98cm2b8',
 'xqzoehu7bk',
 'xpf9r3psp3',
 'kjvtedlnal',
 'vw3jd29zx9',
 'bwggs60n79',
 'nxxinxp6wl',
 'io9kh261t5']

## Parse BibTex

In [11]:
with open('./notebook-va.bib', 'r') as fp:
    lines = [l for l in fp.readlines()]

bibtex_entries = []
cur_entry = ''
for line in lines:
    if line == '\n':
       bibtex_entries.append(cur_entry)
       cur_entry = ''
    
    else:
        cur_entry += line

bibtex_entries.append(cur_entry)

# Create a dictionary that maps bibtex key to the entry
bibtex_dict = {}

for i, entry in enumerate(bibtex_entries):
    first_line = entry.split('\n')[0]
    key = re.sub(r'^@.+{(.+),.*$', r'\1', first_line)
    
    # Also parse the year
    year = int(re.sub(r'^.*(\d{4})[a-z]?$', r'\1', key))
    
    bibtex_dict[key] = {
        'bibtex': entry,
        'year': year
    }

len(bibtex_entries)

193

## Parse Meta Data (Packages + Papers)

In [21]:
with open('./Interesting.html', 'r') as fp:
    content = fp.read()
    soup = BeautifulSoup(content, 'html.parser')
    
package_table = soup.find('table')
package_rows = package_table.find_all('tr')

with open('./Papers.html', 'r') as fp:
    content = fp.read()
    soup = BeautifulSoup(content, 'html.parser')
    
paper_table = soup.find('table')
paper_rows = paper_table.find_all('tr')

def create_new_entry():
    return {
        'name': '',
        'nameDisplay': '',
        'githubURL': '',
        'paperURL': '',
        'otherURLs': [],
        'description': '',
        'bibtex': '',
        'bibtexKey': '',
        'sourceType': '',
        'releaseYear': 0,
        'communication': '',
        'materials': [],
        'layouts': [],
        'supportedNotebooks': [],
        'modularity': '',
        'user': '',
        'implementation': '',
    }

In [22]:
entries = []

# Parse papers first
for i, row in tqdm(enumerate(paper_rows), disable=True):

    columns = row.find_all('td')
    if columns is None or len(columns) == 0:
        continue
    
    entry = create_new_entry()
    entry['sourceType'] = 'paper'
    
    # Get the package name
    name = columns[0]
    display_name = name.text.split('.')[0]
    lower_name = display_name.lower().replace(' ', '-')
    
    entry['name'] = lower_name
    entry['nameDisplay'] = display_name
    
    if lower_name is None or lower_name == '' or lower_name == 'name':
        continue
    
    # Get the paper url
    tags = columns[2]
    tags = tags.find_all('a', recursive=True)
    urls = [tag.get('href') for tag in tags]
    
    for url in urls:
        if entry['paperURL'] == '':
            entry['paperURL'] = url
            
    # Get the description
    description = columns[4].text
    entry['description'] = description
    
    # Get the github url
    tag = columns[5]
    tag = tag.find('a', recursive=True)
    if tag is not None:
        entry['githubURL'] = tag.get('href')
    
    # Get the bibtex
    bibtex_key = columns[3].text
    
    try:
        entry['bibtex'] = bibtex_dict[bibtex_key]['bibtex']
        entry['bibtexKey'] = bibtex_key
        entry['releaseYear'] = bibtex_dict[bibtex_key]['year']
    
    except:
        print(bibtex_key)
        
    # Get the platform
    platforms = [p.replace(' ', '') for p in columns[10].text.split(',')]
    entry['supportedNotebooks'] = platforms
    
    # Get the user
    user = columns[13].text
    entry['user'] = user
    
    # Get the implementation detail
    implementation = columns[15].text
    entry['implementation'] = implementation
    
    # Get the data types
    materials = [p.replace(' ', '') for p in columns[16].text.split(',')]
    entry['materials'] = materials

    # Get the communication style
    communication = columns[17].text
    entry['communication'] = communication
    
    # Get the modularity style
    modularity = columns[18].text
    entry['modularity'] = modularity
    
    # Get the layout style
    layout = columns[19].text
    entry['layouts'] = [layout]
            
    entries.append(entry)

In [23]:
# Prase packages
existing_names = set([entry['name'] for entry in entries])

for i, row in tqdm(enumerate(package_rows), disable=True):

    columns = row.find_all('td')
    if columns is None or len(columns) == 0:
        continue
    
    entry = create_new_entry()
    entry['sourceType'] = 'package'
    
    # Get the package name
    name = columns[0]
    display_name = name.text.split('.')[0]
    display_name = display_name.replace('_', '-')
    lower_name = display_name.lower().replace(' ', '-')
    
    # skip duplicates from papers
    if lower_name in existing_names:
        print('Duplicate: ', lower_name)
        continue

    entry['name'] = lower_name
    entry['nameDisplay'] = display_name
    
    if lower_name is None or lower_name == '' or lower_name == 'module':
        continue
    
    # Get the package urls
    tags = columns[3]
    tags = tags.find_all('a', recursive=True)
    urls = [tag.get('href') for tag in tags]
    
    for url in urls:
        if 'github.com' in url:
            if entry['githubURL'] == '':
                entry['githubURL'] = url
            else:
                entry['otherURLs'].append(url)    
        else:
            entry['otherURLs'].append(url)
            
    # Get the description
    description = columns[4].text
    entry['description'] = description
    
    # Get the bibtex
    bibtex_key = columns[5].text
    
    try:
        entry['bibtex'] = bibtex_dict[bibtex_key]['bibtex']
        entry['bibtexKey'] = bibtex_key
        entry['releaseYear'] = bibtex_dict[bibtex_key]['year']
    
    except:
        print('no bibtex:', bibtex_key)
        
    # Get the platform
    platforms = [p.replace(' ', '') for p in columns[9].text.split(',')]
    entry['supportedNotebooks'] = platforms
    
    # Get the user
    user = columns[12].text
    entry['user'] = user
    
    # Get the implementation detail
    implementation = columns[14].text
    entry['implementation'] = implementation
    
    # Get the data types
    materials = [p.replace(' ', '') for p in columns[15].text.split(',')]
    entry['materials'] = materials

    # Get the communication style
    communication = columns[16].text
    entry['communication'] = communication
    
    # Get the modularity style
    modularity = columns[17].text
    entry['modularity'] = modularity
    
    # Get the layout style
    layout = columns[18].text
    entry['layouts'] = [layout]
            
    entries.append(entry)

Duplicate:  interpretml
Duplicate:  visual-auditor
Duplicate:  pipelineprofiler
Duplicate:  vizseq
Duplicate:  what-if-tool


In [24]:
with open('./resources/supernova.yaml', 'w+') as fp:
    for entry in entries:
        fp.write(yaml.dump([entry]))
        fp.write('\n')

## Generate Table

In [25]:
entries = yaml.safe_load(open('./resources/supernova.yaml', 'r'))
print(len(entries))
entries.sort(key=lambda x: x['name'])

161


In [26]:
latex_table = ""

# icon_map = {
#     "scientist": "icon-user-scientist-alt",
#     "educator": "icon-user-student-alt",
#     "data scientist": "icon-user-data-alt",
#     "monolithic": "icon-mono",
#     "modular": "icon-modular",
#     "on-demand": "icon-display-demand",
#     "always-on": "icon-display-always",
#     "runtime": "icon-data-runtime",
#     "code": "icon-data-text",
#     "external": "icon-data-external",
#     "no": "icon-comm-no",
#     "one-way": "icon-comm-one",
#     "two-way": "icon-comm-two",
#     "jupyter": "icon-plat-jupyter",
#     "jupyter-lab": "icon-plat-jupyter-lab",
#     "lab": "icon-plat-lab",
#     "all": "icon-plat-all",
#     "custom": "icon-impl-custom",
#     "html": "icon-impl-html",
#     "ipywidget": "icon-impl-widget",
#     "extension": "icon-impl-extension",
#     "other-package": "icon-impl-package",
#     "nova": "icon-impl-nova",
# }

icon_map = {
    "scientist": "icon-user-scientist-alt-color",
    "educator": "icon-user-student-alt-color",
    "data scientist": "icon-user-data-alt-color",
    "monolithic": "icon-mono-color",
    "modular": "icon-modular-color",
    "on-demand": "icon-display-demand-color",
    "always-on": "icon-display-always-color",
    "runtime": "icon-data-runtime-color",
    "code": "icon-data-text-color",
    "external": "icon-data-external-color",
    "no": "icon-comm-no-color",
    "one-way": "icon-comm-one-color",
    "two-way": "icon-comm-two-color",
    "jupyter": "icon-plat-jupyter",
    "jupyter-lab": "icon-plat-jupyter-lab",
    "lab": "icon-plat-lab",
    "all": "icon-plat-all",
    "custom": "icon-impl-custom",
    "html": "icon-impl-html",
    "ipywidget": "icon-impl-widget",
    "extension": "icon-impl-extension",
    "other-package": "icon-impl-package",
    "nova": "icon-impl-nova",
}

for i, entry in enumerate(entries):
    cur_row = ""

    if i % 2 == 1:
        cur_row += "\\rowcolor{tablerowcolor}\n"

    # Name
    cur_row += f'{entry["nameDisplay"]}~\citeapp{{{entry["bibtexKey"]}}} & '

    # User
    user = entry["user"]
    cur_row += f"\inlinetablefig{{{icon_map[user]}}} & "

    # Communication
    comm = entry["communication"]
    cur_row += f"\inlinetablefig{{{icon_map[comm]}}} & "

    # Data
    for data in entry["materials"]:
        cur_row += f"\inlinetablefig{{{icon_map[data]}}} "
    cur_row += "&"

    # Context
    layout = entry["layouts"][0]
    cur_row += f"\inlinetablefig{{{icon_map[layout]}}} & "

    # Modularity
    modularity = entry["modularity"]
    cur_row += f"\inlinetablefig{{{icon_map[modularity]}}} & "

    # Platforms
    platforms = entry["supportedNotebooks"]
    if len(platforms) == 3:
        cur_row += f'\inlinetablefig{{{icon_map["all"]}}} & '
    elif len(platforms) == 2:
        cur_row += f'\inlinetablefig{{{icon_map["jupyter-lab"]}}} & '
    else:
        platform = platforms[0]
        cur_row += f"\inlinetablefig{{{icon_map[platform]}}} & "

    # Implementations
    implementation = entry["implementation"]
    cur_row += f"\inlinetablefig{{{icon_map[implementation]}}} \\\\ \n\n"

    latex_table += cur_row

with open('./resources/latex-table.txt', 'w') as fp:
    fp.write(latex_table)